In [1]:
import os
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from find_missing_predictions_util import find_missing_predictions
from lags_util import create_all_advanced_lags
from update_data_util import fetch_and_update_stock_data, update_predictions_file

In [2]:
ticker_to_basename = {
    # Indices
    "^NSEI": "NIFTY_50",
    "^NSEBANK": "NIFTY_BANK",
    "^CNXIT": "NIFTY_IT",
    "^CNXPHARMA": "NIFTY_PHARMA",
    "^CNXFMCG": "NIFTY_FMCG",
    "^CNXAUTO": "NIFTY_AUTO",
    "^CNXMETAL": "NIFTY_METAL",
    "^CNXREALTY": "NIFTY_REALTY",
    "^CNXENERGY": "NIFTY_ENERGY",
    "NIFTY_FIN_SERVICE.NS": "NIFTY_FIN_SERVICE",
    
    # Stocks
    "RELIANCE.NS": "RELIANCE_INDUSTRIES_LTD",
    "TCS.NS": "TATA_CONSULTANCY_SERV_LT", # Matched from your image
    "SUNPHARMA.NS": "SUN_PHARMACEUTICAL_IND_L",
    "ICICIBANK.NS": "ICICI_BANK_LTD.",
    "INFY.NS": "INFOSYS_LIMITED",
    "SBIN.NS": "STATE_BANK_OF_INDIA",
    "BHARTIARTL.NS": "BHARTI_AIRTEL_LIMITED",
    "ITC.NS": "ITC_LTD",
    "LT.NS": "LARSEN_&_TOUBRO_LTD.",
    "HINDUNILVR.NS": "HINDUSTAN_UNILEVER_LTD."
}



raw_data_folder = r"Raw_Data"
predicted_data_folder = r"Pred_Data"
lagging_data_folder = r"Lag_Data"


In [3]:
# Retriving data from Yahoo API
for ticker in ticker_to_basename:
    print(f"\n{'='*80}")
    print(f"📊 Processing: {ticker}")
    try:
        fetch_and_update_stock_data(ticker_symbol=ticker, data_dir=raw_data_folder)
    except Exception as e:
        print(f"❌ Error fetching {ticker}: {e}")


📊 Processing: ^NSEI
📈 Updating existing data for NIFTY_50...
Last record in file is from: 2025-11-17 15:15:00+05:30
✅ Data updated from 2025-11-18 09:15:00+05:30 to 2025-11-20 14:15:00+05:30
📊 Added 20 new records

📊 Processing: ^NSEBANK
📈 Updating existing data for NIFTY_BANK...
Last record in file is from: 2025-11-17 15:15:00+05:30
✅ Data updated from 2025-11-18 09:15:00+05:30 to 2025-11-20 14:15:00+05:30
📊 Added 20 new records

📊 Processing: ^CNXIT
📈 Updating existing data for NIFTY_IT...
Last record in file is from: 2025-11-17 15:15:00+05:30
✅ Data updated from 2025-11-18 09:15:00+05:30 to 2025-11-20 14:15:00+05:30
📊 Added 20 new records

📊 Processing: ^CNXPHARMA
📈 Updating existing data for NIFTY_PHARMA...
Last record in file is from: 2025-11-17 15:15:00+05:30
✅ Data updated from 2025-11-18 09:15:00+05:30 to 2025-11-20 14:15:00+05:30
📊 Added 20 new records

📊 Processing: ^CNXFMCG
📈 Updating existing data for NIFTY_FMCG...
Last record in file is from: 2025-11-17 15:15:00+05:30
✅ D

In [4]:
# updating Prediction file
print(f"\n{'='*80}")
print(f"🔮 Generating predictions for all equities...")

for ticker, basename in ticker_to_basename.items():
    print(f"---> Processing {basename}")
    
    try:
        raw_file_name = f"{basename}_data.csv"
        raw_file_path = os.path.join(raw_data_folder, raw_file_name)
        pred_file_name = f"{basename}_predictions_xgboost.csv"
        pred_file_path = os.path.join(predicted_data_folder, pred_file_name)
        
        update_predictions_file(raw_file_path, pred_file_path)
        print(f"    ✅ Successfully updated: {pred_file_name}")
    
    except Exception as e:
        print(f"    ❌ Error processing prediction for {basename}: {e}")

print("\nAll processing complete.")


🔮 Generating predictions for all equities...
---> Processing NIFTY_50
Appended 20 new entries to Pred_Data\NIFTY_50_predictions_xgboost.csv
    ✅ Successfully updated: NIFTY_50_predictions_xgboost.csv
---> Processing NIFTY_BANK
Appended 20 new entries to Pred_Data\NIFTY_BANK_predictions_xgboost.csv
    ✅ Successfully updated: NIFTY_BANK_predictions_xgboost.csv
---> Processing NIFTY_IT
Appended 20 new entries to Pred_Data\NIFTY_IT_predictions_xgboost.csv
    ✅ Successfully updated: NIFTY_IT_predictions_xgboost.csv
---> Processing NIFTY_PHARMA
Appended 20 new entries to Pred_Data\NIFTY_PHARMA_predictions_xgboost.csv
    ✅ Successfully updated: NIFTY_PHARMA_predictions_xgboost.csv
---> Processing NIFTY_FMCG
Appended 20 new entries to Pred_Data\NIFTY_FMCG_predictions_xgboost.csv
    ✅ Successfully updated: NIFTY_FMCG_predictions_xgboost.csv
---> Processing NIFTY_AUTO
Appended 20 new entries to Pred_Data\NIFTY_AUTO_predictions_xgboost.csv
    ✅ Successfully updated: NIFTY_AUTO_predictions_

In [5]:
# --- Loop Through All Equities to Generate Lag Features ---

print(f"\n{'='*80}")
print(f"📈 Generating lag features for all equities...")
print(f"{'='*80}")

# We just need the "basename" values from the map (e.g., "NIFTY_50")
for basename in ticker_to_basename.values():
    print(f"---> Processing {basename}")
    
    try:
        # 1. Construct the path to the prediction file (Input)
        #    (This assumes the output from your previous script)
        pred_file_name = f"{basename}_predictions_xgboost.csv"
        pred_file_path = os.path.join(predicted_data_folder, pred_file_name)

        # 2. Construct the path for the new lag file (Output)
        lag_file_name = f"{basename}_lagged.csv"
        lag_file_path = os.path.join(lagging_data_folder, lag_file_name)
        
        # 3. Read the prediction data
        data = pd.read_csv(pred_file_path, parse_dates=['Datetime'], index_col='Datetime')
        if data.index.tz is not None:
            data.index = data.index.tz_localize(None)

        # 4. Isolate the 'ActualPrice' column as the base for lags
        data_df = data[['ActualPrice']].copy()

        # 5. Create the advanced lag features
        full_lagged = create_all_advanced_lags(data_df)
        if full_lagged.index.tz is not None:
            full_lagged.index = full_lagged.index.tz_localize(None)

        # 6. Save the new lagged data to the Lag_Data folder
        full_lagged.to_csv(lag_file_path)
        print(f"    ✅ Successfully created: {lag_file_name}")

    except FileNotFoundError:
        print(f"    ⚠️  Skipping {basename}: Prediction file not found at {pred_file_path}")
    except Exception as e:
        print(f"    ❌ Error processing lags for {basename}: {e}")

print("\nAll lag feature generation complete.")


📈 Generating lag features for all equities...
---> Processing NIFTY_50
    ✅ Successfully created: NIFTY_50_lagged.csv
---> Processing NIFTY_BANK
    ✅ Successfully created: NIFTY_BANK_lagged.csv
---> Processing NIFTY_IT
    ✅ Successfully created: NIFTY_IT_lagged.csv
---> Processing NIFTY_PHARMA
    ✅ Successfully created: NIFTY_PHARMA_lagged.csv
---> Processing NIFTY_FMCG
    ✅ Successfully created: NIFTY_FMCG_lagged.csv
---> Processing NIFTY_AUTO
    ✅ Successfully created: NIFTY_AUTO_lagged.csv
---> Processing NIFTY_METAL
    ✅ Successfully created: NIFTY_METAL_lagged.csv
---> Processing NIFTY_REALTY
    ✅ Successfully created: NIFTY_REALTY_lagged.csv
---> Processing NIFTY_ENERGY
    ✅ Successfully created: NIFTY_ENERGY_lagged.csv
---> Processing NIFTY_FIN_SERVICE
    ✅ Successfully created: NIFTY_FIN_SERVICE_lagged.csv
---> Processing RELIANCE_INDUSTRIES_LTD
    ✅ Successfully created: RELIANCE_INDUSTRIES_LTD_lagged.csv
---> Processing TATA_CONSULTANCY_SERV_LT
    ✅ Successfully 

In [6]:
# Modelling 

for equity_ticker_symbol, equity in ticker_to_basename.items():
    print(f"\n🚀 Processing {equity_ticker_symbol} ({equity}) ...")

    try:
        raw_data_file = os.path.join(raw_data_folder, f"{equity}_data.csv")
        predicted_data_file = os.path.join(predicted_data_folder, f"{equity}_predictions_xgboost.csv")
        lagging_data_file = os.path.join(lagging_data_folder, f"{equity}_lagged.csv")

        raw_data = pd.read_csv(raw_data_file, parse_dates=['Datetime'], index_col='Datetime')
        predicted_data = pd.read_csv(predicted_data_file, parse_dates=['Datetime'], index_col='Datetime')
        lagging_data = pd.read_csv(lagging_data_file, parse_dates=['Datetime'], index_col='Datetime')

        print(f"✅ Files loaded for {equity} | Raw: {len(raw_data)} | Lagged: {len(lagging_data)} | Pred: {len(predicted_data)}")

        num_missing, missing_rows_df = find_missing_predictions(predicted_data_file)

        if num_missing == 0:
            print(f"✅ All predictions are already up-to-date for {equity}.")
            continue

        print(f"⚠️ Found {num_missing} missing predictions. Starting update...")

        # --- Determine starting datetime ---
        first_missing_datetime = pd.to_datetime(missing_rows_df.index[0])
        first_lagged_datetime = pd.to_datetime(lagging_data.index[0])
        start_datetime = max(first_missing_datetime, first_lagged_datetime)
        start_n = lagging_data.index.get_loc(start_datetime)

        print(f"Starting prediction from index {start_n} | datetime: {start_datetime}")

        # --- Prediction loop ---
        for i, n in enumerate(range(start_n, len(lagging_data)), 1):
            # Prepare training data
            if n > 2000:
                train_lagged = lagging_data.iloc[n-2000:n]
            else:
                train_lagged = lagging_data.iloc[:n]

            X_train = train_lagged.drop(columns=['ActualPrice'])
            y_train = train_lagged['ActualPrice']

            # Train model
            model = XGBRegressor(
                n_estimators=650,
                random_state=42,
                colsample_bytree=0.8,
                learning_rate=0.06,
                max_depth=6,
                n_jobs=-1
            )
            model.fit(X_train, y_train)

            # Predict next point
            test_point = lagging_data.iloc[n]
            X_test_point = test_point.drop('ActualPrice').to_frame().T
            y_pred = model.predict(X_test_point)[0]

            # Store prediction
            pred_index = lagging_data.index[n]
            if pred_index in predicted_data.index:
                predicted_data.at[pred_index, 'PredictedPrice'] = y_pred

            # Save every 20 steps
            if i % 20 == 0:
                predicted_data.to_csv(predicted_data_file)
                print(f"💾 Saved {i} new predictions so far... ({equity})")

        # --- Final save ---
        predicted_data.to_csv(predicted_data_file)
        print(f"All {num_missing} missing predictions saved for {equity}.")

        # --- Predict final point (sanity check) ---
        train_lagged = lagging_data.iloc[-2000:-1] if len(lagging_data) > 2000 else lagging_data.iloc[:-1]
        X_train = train_lagged.drop(columns=['ActualPrice'])
        y_train = train_lagged['ActualPrice']

        model.fit(X_train, y_train)
        last_point = lagging_data.iloc[-1].drop('ActualPrice').to_frame().T
        last_pred = model.predict(last_point)[0]
        print(f"🔮 Final prediction for {equity}: {last_pred:.2f} at {lagging_data.index[-1]}")

    except Exception as e:
        print(f"❌ Error processing {equity}: {str(e)}")
        continue

print("\n🎉 All equities processed successfully!")



🚀 Processing ^NSEI (NIFTY_50) ...
✅ Files loaded for NIFTY_50 | Raw: 3942 | Lagged: 3934 | Pred: 3942
Found 20 rows with missing 'PredictedPrice' values.
⚠️ Found 20 missing predictions. Starting update...
Starting prediction from index 3914 | datetime: 2025-11-18 09:15:00
💾 Saved 20 new predictions so far... (NIFTY_50)
All 20 missing predictions saved for NIFTY_50.
🔮 Final prediction for NIFTY_50: 26236.27 at 2025-11-20 14:15:00

🚀 Processing ^NSEBANK (NIFTY_BANK) ...
✅ Files loaded for NIFTY_BANK | Raw: 3942 | Lagged: 3934 | Pred: 3942
Found 20 rows with missing 'PredictedPrice' values.
⚠️ Found 20 missing predictions. Starting update...
Starting prediction from index 3914 | datetime: 2025-11-18 09:15:00
💾 Saved 20 new predictions so far... (NIFTY_BANK)
All 20 missing predictions saved for NIFTY_BANK.
🔮 Final prediction for NIFTY_BANK: 59361.16 at 2025-11-20 14:15:00

🚀 Processing ^CNXIT (NIFTY_IT) ...
✅ Files loaded for NIFTY_IT | Raw: 3521 | Lagged: 3513 | Pred: 3521
Found 20 rows

In [7]:
# Return Calculation Loop

for equity_ticker_symbol, equity in ticker_to_basename.items():
    print(f"\n📊 Processing {equity_ticker_symbol} ({equity}) ...")

    try:
        predicted_data_file = os.path.join(predicted_data_folder, f"{equity}_predictions_xgboost.csv")

        # Skip missing files
        if not os.path.exists(predicted_data_file):
            print(f"⚠️ File not found: {predicted_data_file}")
            continue

        # Load data
        predicted_data = pd.read_csv(predicted_data_file, parse_dates=['Datetime'], index_col='Datetime')

        print(f"Loaded {len(predicted_data)} rows from {predicted_data_file}")

        # --- Check required columns ---
        required_cols = {'PredictedPrice', 'ActualPrice'}
        if not required_cols.issubset(predicted_data.columns):
            print(f"❌ Missing columns in {equity}: {required_cols - set(predicted_data.columns)}")
            continue

        # --- Compute or fill 'return' column ---
        if 'return' not in predicted_data.columns:
            predicted_data['return'] = (
                (predicted_data['PredictedPrice'] - predicted_data['ActualPrice']) / predicted_data['ActualPrice']
            )
            print(f"🆕 'return' column created for {equity}.")
        else:
            missing_mask = predicted_data['return'].isna()
            if missing_mask.any():
                predicted_data.loc[missing_mask, 'return'] = (
                    (predicted_data.loc[missing_mask, 'PredictedPrice'] - predicted_data.loc[missing_mask, 'ActualPrice'])
                    / predicted_data.loc[missing_mask, 'ActualPrice']
                )
                print(f"♻️ Filled {missing_mask.sum()} missing 'return' values for {equity}.")
            else:
                print(f"✅ 'return' column already complete for {equity}.")

        # --- Save updated file ---
        predicted_data.to_csv(predicted_data_file, index=True)
        print(f"💾 Saved updated data to: {predicted_data_file}")

    except Exception as e:
        print(f"❌ Error processing {equity}: {str(e)}")
        continue

print("\n🎉 All equities processed for 'return' column update successfully!")



📊 Processing ^NSEI (NIFTY_50) ...
Loaded 3942 rows from Pred_Data\NIFTY_50_predictions_xgboost.csv
♻️ Filled 20 missing 'return' values for NIFTY_50.
💾 Saved updated data to: Pred_Data\NIFTY_50_predictions_xgboost.csv

📊 Processing ^NSEBANK (NIFTY_BANK) ...
Loaded 3942 rows from Pred_Data\NIFTY_BANK_predictions_xgboost.csv
♻️ Filled 20 missing 'return' values for NIFTY_BANK.
💾 Saved updated data to: Pred_Data\NIFTY_BANK_predictions_xgboost.csv

📊 Processing ^CNXIT (NIFTY_IT) ...
Loaded 3521 rows from Pred_Data\NIFTY_IT_predictions_xgboost.csv
♻️ Filled 20 missing 'return' values for NIFTY_IT.
💾 Saved updated data to: Pred_Data\NIFTY_IT_predictions_xgboost.csv

📊 Processing ^CNXPHARMA (NIFTY_PHARMA) ...
Loaded 3519 rows from Pred_Data\NIFTY_PHARMA_predictions_xgboost.csv
♻️ Filled 20 missing 'return' values for NIFTY_PHARMA.
💾 Saved updated data to: Pred_Data\NIFTY_PHARMA_predictions_xgboost.csv

📊 Processing ^CNXFMCG (NIFTY_FMCG) ...
Loaded 3521 rows from Pred_Data\NIFTY_FMCG_predicti